# 用户指南 第29章 异音连音符`Slur`
[原文链接](https://web.mit.edu/music21/doc/usersGuide/usersGuide_29_spanners.html)
> 译者注：同音连音符`Tie`见[第31章](31.ipynb)。
> 
> 同音连音符与异音连音符意义不同。同音连音符仅为方便记谱，在演奏与处理时等同于一个音符。异音连音符为演奏技法符号，处理时视为两个音符，人声演唱时一个字跨两个音高，即歌声合成软件中的减号`-`。